<a href="https://colab.research.google.com/github/nataliawojak/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

%load_ext tensorboard

from hyperopt import hp,STATUS_OK, tpe, Trials, fmin

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:

train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:

if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):

  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(X_train,
            y_train,
            batch_size=params_fit.get('batch_size', 128),
            epochs = params_fit.get('epochs', 5),
            verbose=params_fit.get('verbose', 1),
            validation_data=params_fit.get('validation_data', (X_train, y_train)),
            callbacks=[tensorboard_callback]

            
            
            
            
            )
  return model



In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_predict_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_predict_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax'),
])
  


In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 17ms/step - loss: 2.6814 - accuracy: 0.2825 - val_loss: 1.2480 - val_accuracy: 0.5838
Epoch 2/5
272/272 [==============================] - 4s 16ms/step - loss: 1.0817 - accuracy: 0.6496 - val_loss: 0.4060 - val_accuracy: 0.8745
Epoch 3/5
272/272 [==============================] - 4s 16ms/step - loss: 0.5462 - accuracy: 0.8274 - val_loss: 0.1414 - val_accuracy: 0.9575
Epoch 4/5
272/272 [==============================] - 4s 16ms/step - loss: 0.3546 - accuracy: 0.8898 - val_loss: 0.0845 - val_accuracy: 0.9762
Epoch 5/5
272/272 [==============================] - 4s 16ms/step - loss: 0.2607 - accuracy: 0.9195 - val_loss: 0.0444 - val_accuracy: 0.9884


0.9619047619047619

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_two']),

  Dense(num_classes, activation='softmax'),
])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(X_train,
            y_train,
            batch_size=int(params.get('batch_size', 128)),
            epochs = 5,
            verbose=0)
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK,'model': model}


In [14]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
    
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.4633591448385018, 'dropout_cnn_block_three': 0.3927922248519149, 'dropout_cnn_block_two': 0.3069666286394283, 'dropout_dense_block_one': 0.6670117560155427, 'dropout_dense_block_two': 0.5775153555533963}
accuracy=0.9251700639724731
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.4377713260482651, 'dropout_cnn_block_three': 0.4371828384684936, 'dropout_cnn_block_two': 0.48597625402708283, 'dropout_dense_block_one': 0.45828822351982484, 'dropout_dense_block_two': 0.5275155071494704}
accuracy=0.9453514814376831
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.3220652576007554, 'dropout_cnn_block_three': 0.34775746849347805, 'dropout_cnn_block_two': 0.390875830723645, 'dropout_dense_block_one': 0.39798391620697904, 'dropout_dense_block_two': 0.35337137906736726}
accuracy=0.963718831539154
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.3485192177038281, 'dropout_cnn_block_three': 0.4682196181297759, 'dropout_cnn_block_two': 0.3258857280861